# Design 100 qubits programmatically

#### Prerequisite
A working local installation of Ansys

## N_x by N_y grid of qubits

A simple example of generating a large grid of qubits.

Using TransmonPocket and RouteMeander, we can generate a large, varied grid array of qubits. This is not a practical design given the lack of readouts or control lines to the qubits, just an example for how to make a design algorithmically.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, MetalGUI
from qiskit_metal import Dict, Headings

In [ ]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [ ]:
design.overwrite_enabled = True

## Design

We can vary how big we want the grid to be by changing N_x/N_y (number of qubits along the x/y axis).
Be careful as very large arrays can take a fair bit of time to generate. We modify the chip size so it contains all of the qubits.

In [ ]:
N_x =4
N_y =3

In [ ]:
design.chips.main.size.size_x = str((N_x+1) * 3)+'mm'
design.chips.main.size.size_y = str((N_y+1) * 3)+'mm'

design.chips.main.size.center_x = str((N_x-1) * 1.5)+'mm'
design.chips.main.size.center_y = str((N_y-0.5) * 1.5)+'mm'

First we generate the qubits. We use some simple math to generate the offset pattern in order to make the bus resonators easier to connect.

In [ ]:
#Loop to generate and draw the qubits
for x in range(N_x):
    for y in range(N_y):
        options = dict(pos_x= str(x*3000)+'um', pos_y = str(y*3000 + (x%2)*1500)+'um', orientation = "-90",
                      connection_pads = dict(
                            B0 = dict(loc_W=-1, loc_H=-1, pad_width='75um'),
                            B1  = dict(loc_W=-1, loc_H=+1, pad_width='120um'),
                            B2  = dict(loc_W=+1, loc_H=-1, pad_width='120um'),
                            B3 = dict(loc_w = +1, loc_H = +1, pad_width='90um')))

        
        obj=TransmonPocket(design,'Q_'+str(x)+'_'+str(y),options)
        
        
gui.rebuild()
gui.autoscale()

Next, we generate the route meanders. As we used consistent naming schemes for the qubits and pins, we can loop through them with out issue.
We also have the length vary based on which qubit is being connected, such that no qubit should be connected to two resonators of the same frequency.

In [ ]:
for x in range(N_x):
    for y in range(N_y):
        #"upward" connection, avoids drawing connectors for 'top' row. Changes connector length by +/-50um to avoid frequency collisions
        if y<(N_y-1):
            connectorAD = RouteMeander(design,'CU_'+str(x)+'_'+str(y),options = dict(total_length = str(7+(y%2)*0.5)+'mm',
                                                                                    fillet = "99um",lead=dict(
                                                                                    start_straight='0.5mm',
                                                                                    end_straight='0.25mm'),
                                                                                    meander = dict(asymmetry='-700um'),
                                                                                    pin_inputs = dict(
                                                                                        start_pin=dict(
                                                                                            component ='Q_'+str(x)+'_'+str(y),
                                                                                            pin = 'B0'), 
                                                                                        end_pin=dict(
                                                                                            component='Q_'+str(x)+'_'+str(y+1),
                                                                                            pin='B3'))))
            
        #"sideways" connection, avoids drawing for far right col, and for top qubit in odd col. Changes connector length by +/- 25um 
        #to avoid frequency collisions
        if x<(N_x-1) and (not(x%2 and y==(N_y-1))):
            connectorBC = RouteMeander(design,'CS_'+str(x)+'_'+str(y),options= dict(total_length = str(6+(y%2)*0.5)+'mm',
                                                                                    fillet = "99um",lead=Dict(
                                                                                        start_straight='0.3mm',
                                                                                        end_straight='0.25mm'),
                                                                                    meander = Dict(asymmetry='-200um'),
                                                                                    pin_inputs = Dict(
                                                                                        start_pin=Dict(
                                                                                            component ='Q_'+str(x)+'_'+str(y),
                                                                                            pin = 'B1'), 
                                                                                        end_pin=Dict(
                                                                                            component='Q_'+str(x+1)+'_'+str(y+(x%2)),
                                                                                             pin='B2'))))
            

gui.rebuild()
gui.autoscale()

In [ ]:
gui.screenshot()

## Analysis

We can quickly check if we have designed the qubits well with an LOM analysis. 
First we select the design that we want to analyse and the tool to use for any simulation.

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

You can review and update the Analysis default setup following the examples in the next two cells.

In [ ]:
c1.sim.setup

Let's change the name of the setup and increase the maximum number of passes.

In [ ]:
c1.sim.setup.name = 'LOM'
c1.sim.setup.max_passes = 14

#To change multiple settings use the following method:
#c1.sim.setup_update(name = 'LOM', max_passes = 14)

c1.sim.setup


For our analysis we will pick the first qubit, Q_0_0, making sure to apply open terminations to the connection pads. The default bounding box is sufficient for a simple analysis. By passing the parameter `components` to the `sim.run()` method, the design is rendered automatically. Not passing it skips the rendering and tries to run the analysis on the latest design. If a design is not found, the full metal design is rendered.

In [ ]:
c1.sim.run(components=['Q_0_0'], open_terminations=[('Q_0_0', 'B0'), ('Q_0_0', 'B1'), ('Q_0_0', 'B2'), ('Q_0_0', 'B3')])

We then use the results of that simulation to complete a LOM analysis. The input values should be modified based on the simulation passes and the set frequencies of the resonators.

In [ ]:
c1.setup.junctions = Dict({'Lj': 12.31, 'Cj': 2})
c1.setup.freq_readout = 6.6
c1.setup.freq_bus = [6.0, 6.2,6.4]

c1.run_lom()

Once you are done with your analysis, please close it with `close()`. This will free up resources currently occupied by qiskit-metal to communiate with the tool.

In [ ]:
c1.sim.close()

## GDS Render

In [ ]:
chip_gds = design.renderers.gds

In [ ]:
chip_gds.options['no_cheese']['buffer'] = '50um'
chip_gds.options['path_filename'] = '../../resources/Fake_Junctions.GDS'

In [ ]:
chip_gds.export_to_gds("NxN_Chip.gds")

We can close the GUI now that we have finished the design.

In [ ]:
gui.main_window.close()